In [1]:
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import plotly.express as px
import pytorch_lightning as pl
from pl_bolts.models.self_supervised.simclr import SimCLREvalDataTransform
from PIL import Image
import torchmetrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
import tifffile as tiff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.data.dataset import ICDARDataset
from glob import glob
import os

/home/hpc/iwfa/iwfa028h/.conda/envs/lme/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/hpc/iwfa/iwfa028h/.conda/envs/lme/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/hpc/iwfa/iwfa028h/.conda/envs/lme/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io

In [5]:
def data_factory(dataset_name, root_dir, label_filepath, transforms, mode, batch_size, collate_fn=None, num_cpus=None):

    if dataset_name.lower() == 'icdar':
        dataset = ICDARDataset(label_filepath, root_dir, transforms=transforms, convert_rgb=False)
    elif dataset_name.lower() == 'icdar_lightly':
        dataset = LightlyDataset(input_dir=root_dir, transform=transforms, filenames=glob(root_dir + '/*.tif'))
    else:
        raise NotImplementedError(f'Dataset {dataset_name} is not implemented')

    total_count = len(dataset)
    train_count = int(0.7 * total_count)
    val_count = int(0.1 * total_count)
    test_count = total_count - train_count - val_count

    train_dataset, val_dataset, test_dataset = random_split(
        dataset,
        (train_count, val_count, test_count)
    )

    if mode in 'train':
        return {
            'train': DataLoader(
                train_dataset,
                batch_size=batch_size,
                shuffle=True,
                drop_last=True,
                pin_memory=False,
                num_workers=num_cpus or os.cpu_count(),
                collate_fn=collate_fn if collate_fn else None
            ),
            'val': DataLoader(
                val_dataset,
                batch_size=batch_size,
                shuffle=False,
                drop_last=False,
                pin_memory=False,                
                num_workers=num_cpus or os.cpu_count(),
                collate_fn=collate_fn if collate_fn else None
            )
        }
    elif mode == 'test':
        return {
            'test': DataLoader(
                test_dataset,
                batch_size=batch_size,
                shuffle=False,
                drop_last=False,
                pin_memory=False,
                num_workers=os.cpu_count(),
                collate_fn=collate_fn if collate_fn else None
            )
        }
    else:
        raise KeyError(f'Unknown mode: {mode}')


In [6]:
root_dir = os.path.join(
    '/home/woody/iwfa/iwfa028h/dev/faps', 'data', 'ICDAR2017_CLaMM_Training'
)
dataloaders = data_factory(
    dataset_name='icdar',
    root_dir=root_dir, 
    label_filepath=os.path.join(root_dir, '@ICDAR2017_CLaMM_Training.csv'),
    transforms=T.Compose([T.RandomResizedCrop(1200), T.ToTensor()]),
    mode='train',
    batch_size=16,
    num_cpus=4
)

# Calculate the mean and standard deviation
mean = 0.0
std = 0.0
total_samples = 0

for images, _ in dataloaders.get('train'):
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_samples += batch_samples

mean /= total_samples
std /= total_samples

print("Mean:", mean)
print("Standard Deviation:", std)


Mean: tensor([0.7046, 0.7046, 0.7046])
Standard Deviation: tensor([0.2509, 0.2509, 0.2509])


In [11]:
# icdar_mean = [0.7036, 0.7036, 0.7036]
# icdar_std = [0.2501, 0.2501, 0.2501]
icdar_mean=[0.485, 0.456, 0.406]
icdar_std=[0.229, 0.224, 0.225]

fig = make_subplots(rows=2, cols=2, row_heights=[400, 400])

t = T.Compose([T.Normalize(icdar_mean, icdar_std)])

# Load the data
images = []
count = 0
for batch, _ in dataloaders.get('train'):
    if count > 10:
        break
    images.extend(batch[0])
    count += 1

# Create subplots with two images per row
num_images = len(images)
num_rows = (num_images + 1) // 2
fig = make_subplots(rows=num_rows, cols=2)

# Add images to the subplots
for i, image in enumerate(images):
    row = (i // 2) + 1
    col = (i % 2) + 1

    # Convert the tensor image to a numpy array
    image = t(image).permute(1, 2, 0).numpy()  # Assuming image shape is (C, H, W)

    # Add the image to the subplot
    fig.add_trace(go.Image(z=image), row=row, col=col)

# Configure subplot layout
fig.update_layout(showlegend=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

# Show the figure
fig.show()

ValueError: Expected tensor to be a tensor image of size (..., C, H, W). Got tensor.size() = torch.Size([1200, 1200])